<a href="https://colab.research.google.com/github/lasiadhi/Prediction-of-hypoxemia-trend/blob/master/Labs_and_vitals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#################################################
###### @author: Lasith Adhikari (python 3)  #####
############ Created on Nov 21, 2019 ############

In [0]:
# Import libraries
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.path as path
import tensorflow as tf

# Below imports are used to print out pretty pandas dataframes
from IPython.display import display, HTML

# Imports for accessing Datathon data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

In [0]:
from google.colab import files
import seaborn as sns
import pickle
import matplotlib.pyplot as plt 
pd.options.display.max_rows = 200

In [3]:

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# read vitals:
vitals_df = pickle.load(open("/content/drive/My Drive/Colab Notebooks/data/pivoted_vital_bin.pkl", "rb" ) )

In [14]:
vitals_df.head(100)

,id,bin,min_t_heartrate,max_t_heartrate,med_t_heartrate,avg_t_heartrate,sd_t_heartrate,min_t_respiratoryrate,max_t_respiratoryrate,med_t_respiratoryrate,avg_t_respiratoryrate,sd_t_respiratoryrate,min_t_spo2,max_t_spo2,med_t_spo2,avg_t_spo2,sd_t_spo2,min_t_nibp_systolic,max_t_nibp_systolic,med_t_nibp_systolic,avg_t_nibp_systolic,sd_t_nibp_systolic,min_t_nibp_diastolic,max_t_nibp_diastolic,med_t_nibp_diastolic,avg_t_nibp_diastolic,sd_t_nibp_diastolic,min_t_nibp_mean,max_t_nibp_mean,med_t_nibp_mean,avg_t_nibp_mean,sd_t_nibp_mean,min_t_temperature,max_t_temperature,med_t_temperature,avg_t_temperature,sd_t_temperature,min_t_ibp_systolic,max_t_ibp_systolic,med_t_ibp_systolic,avg_t_ibp_systolic,sd_t_ibp_systolic,min_t_ibp_diastolic,max_t_ibp_diastolic,med_t_ibp_diastolic,avg_t_ibp_diastolic,sd_t_ibp_diastolic,min_t_ibp_mean,max_t_ibp_mean,med_t_ibp_mean,avg_t_ibp_mean,sd_t_ibp_mean
0,141168,1,101.0,104.0,102.0,102.333333,1.527525,NaN,NaN,NaN,NaN,NaN,96.0,96.0,96.0,96.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.9,35.9,35.90,35.900000,NaN,112.0,124.0,118.0,118.0,8.485281,56.0,62.0,59.0,59.0,4.242641,80.0,88.0,84.0,84.000000,5.656854
1,141168,2,92.0,92.0,92.0,92.000000,NaN,NaN,NaN,NaN,NaN,NaN,98.0,98.0,98.0,98.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,141168,3,78.0,78.0,78.0,78.000000,NaN,NaN,NaN,NaN,NaN,NaN,97.0,97.0,97.0,97.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,141168,4,72.0,72.0,72.0,72.000000,NaN,NaN,NaN,NaN,NaN,NaN,98.0,98.0,98.0,98.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,141168,5,74.0,74.0,74.0,74.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,141168,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96.0,96.0,96.0,96.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,141168,7,66.0,66.0,66.0,66.000000,NaN,NaN,NaN,NaN,NaN,NaN,90.0,90.0,90.0,90.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,141168,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94.0,94.0,94.0,94.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,141168,9,76.0,76.0,76.0,76.000000,NaN,NaN,NaN,NaN,NaN,NaN,96.0,96.0,96.0,96.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,141168,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
vitals_df.id.nunique()

38823

In [23]:
# check for the availability of bin data:
round(np.sum(vitals_df.groupby('id').count()>=1)/vitals_df.id.nunique()*100,2)

bin                      100.00
min_t_heartrate           79.87
max_t_heartrate           79.87
med_t_heartrate           79.87
avg_t_heartrate           79.87
sd_t_heartrate            56.09
min_t_respiratoryrate     76.50
max_t_respiratoryrate     76.50
med_t_respiratoryrate     76.50
avg_t_respiratoryrate     76.50
sd_t_respiratoryrate      51.89
min_t_spo2                69.66
max_t_spo2                69.66
med_t_spo2                69.66
avg_t_spo2                69.66
sd_t_spo2                 48.68
min_t_nibp_systolic       73.26
max_t_nibp_systolic       73.26
med_t_nibp_systolic       73.26
avg_t_nibp_systolic       73.26
sd_t_nibp_systolic        45.70
min_t_nibp_diastolic      73.25
max_t_nibp_diastolic      73.25
med_t_nibp_diastolic      73.25
avg_t_nibp_diastolic      73.25
sd_t_nibp_diastolic       45.69
min_t_nibp_mean           64.32
max_t_nibp_mean           64.32
med_t_nibp_mean           64.32
avg_t_nibp_mean           64.32
sd_t_nibp_mean            38.26
min_t_te

In [0]:
# read labs:
labs_df = pickle.load(open("/content/drive/My Drive/Colab Notebooks/data/pivoted_lab_bin.pkl", "rb" ) )

In [17]:
labs_df.head(100)

,id,bin,min_t_albumin,max_t_albumin,med_t_albumin,avg_t_albumin,sd_t_albumin,min_t_bilirubin,max_t_bilirubin,med_t_bilirubin,avg_t_bilirubin,sd_t_bilirubin,min_t_BUN,max_t_BUN,med_t_BUN,avg_t_BUN,sd_t_BUN,min_t_calcium,max_t_calcium,med_t_calcium,avg_t_calcium,sd_t_calcium,min_t_creatinine,max_t_creatinine,med_t_creatinine,avg_t_creatinine,sd_t_creatinine,min_t_glucose,max_t_glucose,med_t_glucose,avg_t_glucose,sd_t_glucose,min_t_bicarbonate,max_t_bicarbonate,med_t_bicarbonate,avg_t_bicarbonate,sd_t_bicarbonate,min_t_TotalCO2,max_t_TotalCO2,med_t_TotalCO2,...,min_t_potassium,max_t_potassium,med_t_potassium,avg_t_potassium,sd_t_potassium,min_t_ptt,max_t_ptt,med_t_ptt,avg_t_ptt,sd_t_ptt,min_t_sodium,max_t_sodium,med_t_sodium,avg_t_sodium,sd_t_sodium,min_t_wbc,max_t_wbc,med_t_wbc,avg_t_wbc,sd_t_wbc,min_t_bands,max_t_bands,med_t_bands,avg_t_bands,sd_t_bands,min_t_alt,max_t_alt,med_t_alt,avg_t_alt,sd_t_alt,min_t_ast,max_t_ast,med_t_ast,avg_t_ast,sd_t_ast,min_t_alp,max_t_alp,med_t_alp,avg_t_alp,sd_t_alp
0,141168,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,141168,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,141168,3,3.0,3.0,3.0,3.0,NaN,5.2,5.2,5.2,5.2,NaN,29.0,29.0,29.0,29.0,NaN,8.5,8.5,8.5,8.5,NaN,2.95,2.95,2.95,2.95,NaN,54.0,54.0,54.0,54.0,NaN,18.0,18.0,18.0,18.0,NaN,NaN,NaN,NaN,...,4.3,4.3,4.3,4.3,NaN,36.0,36.0,36.0,36.0,NaN,140.0,140.0,140.0,140.0,NaN,19.8,19.8,19.8,19.8,NaN,NaN,NaN,NaN,NaN,NaN,1189.0,1189.0,1189.0,1189.0,NaN,4257.0,4257.0,4257.0,4257.0,NaN,138.0,138.0,138.0,138.0,NaN
3,141168,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,141168,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,141168,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,141168,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,141168,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,141168,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [18]:
labs_df.id.nunique()

38823

In [24]:
# check for the availability of bin data:
round(np.sum(labs_df.groupby('id').count()>=1)/labs_df.id.nunique()*100,2)

bin                  100.00
min_t_albumin         39.06
max_t_albumin         39.06
med_t_albumin         39.06
avg_t_albumin         39.06
sd_t_albumin           0.06
min_t_bilirubin       36.34
max_t_bilirubin       36.34
med_t_bilirubin       36.34
avg_t_bilirubin       36.34
sd_t_bilirubin         0.05
min_t_BUN             64.00
max_t_BUN             64.00
med_t_BUN             64.00
avg_t_BUN             64.00
sd_t_BUN               1.26
min_t_calcium         60.83
max_t_calcium         60.83
med_t_calcium         60.83
avg_t_calcium         60.83
sd_t_calcium           0.52
min_t_creatinine      64.19
max_t_creatinine      64.19
med_t_creatinine      64.19
avg_t_creatinine      64.19
sd_t_creatinine        1.40
min_t_glucose         80.88
max_t_glucose         80.88
med_t_glucose         80.88
avg_t_glucose         80.88
sd_t_glucose          24.87
min_t_bicarbonate     60.91
max_t_bicarbonate     60.91
med_t_bicarbonate     60.91
avg_t_bicarbonate     60.91
sd_t_bicarbonate    